## Генерация текста с помощью LSTM

Занятие номер восемь курса ["Основы нейронных сетей на Python"](https://neural-university.ru/neural-kurs).

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

In [1]:
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import sys
import io
from google.colab import drive
from google.colab import files

Using TensorFlow backend.


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
print(tf.__version__)

1.13.1


In [0]:
pip install tensorflow-gpu==2.0.0-alpha0

## Загружаем данные для обучения

Тексты Ницше (на английском)

In [0]:
# !wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [0]:
# !head nietzsche.txt

Текст Шекспира (на английском)

In [0]:
# !wget http://www.gutenberg.org/files/100/100-0.txt

In [0]:
# !head 100-0.txt

Текст Войны и Мира (на русском)

In [0]:
# !find . -type f -name "corpus.ru (1).txt" -delete
# !ls

In [0]:
# files.upload()

In [0]:
# !unzip war_and_peace.zip

In [0]:
# !head war_and_peace.txt  

In [0]:
# files.upload()

In [0]:
!ls

Читаем данные в память

In [0]:
drive.mount('/content/gdrive')

In [0]:
# Ницше
# path = 'nietzsche.txt'
# Шекспир
# path = '100-0.txt'
# Толстой
# path = 'gdrive/My Drive/saved_nnets/war_and_peace.h.txt'
# Русский корпус
path = '/content/gdrive/My Drive/saved_nnets/corpus.ru.strict.half.txt'

In [0]:
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

In [0]:
text[:50]

Получаем информацию о символах в тексте

In [0]:
chars = sorted(list(set(text)))

In [0]:
chars

Готовим словари индексов для символов

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
char_indices

In [0]:
indices_char

## Разбиваем текст на последовательности строк для обучения

В статье "A survey" прочитал, что достаточно знать предыдущие 8 символов - ?

In [0]:
context_length = 40
step = 20
sentences = []
next_chars = []

In [0]:
for i in range(0, len(text) - context_length, step):
    sentences.append(text[i: i + context_length])
    next_chars.append(text[i + context_length])

In [0]:
sentences[1]

In [0]:
next_chars[1]

## Векторизация текста в формате One Hot Encoding

In [0]:
x = np.zeros((len(sentences), context_length, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
#нужно преобразовать
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y
  
Y = convert_to_one_hot(Y, num_classes).T

In [0]:
x[0,0]

In [0]:
x[0,1]

In [0]:
y[0]

## Создаем нейронную сеть

In [0]:
def create_model(context_length, char_num):
    model = Sequential()
    model.add(LSTM(128, input_shape=(context_length, char_num)))
    model.add(Dense(char_num, activation='softmax'))
   
    return model

In [0]:
model = create_model(context_length, len(chars))

## Подключаем Google Drive

Обучение занимает много времени. Поэтому будем сохранять нейронную сеть на каждой эпохе на Google Drive

In [0]:
# drive.mount('/content/gdrive')

In [0]:
# !mkdir -p /content/gdrive/'My Drive'/saved_nnets/char_rnn
# !mkdir -p /content/gdrive/'My Drive'/saved_nnets/war_and_peace
# !mkdir -p /content/gdrive/'My Drive'/saved_nnets/war_and_peace_h
# !mkdir -p /content/gdrive/'My Drive'/saved_nnets/ru_corpora
!mkdir -p /content/gdrive/'My Drive'/saved_nnets/ru_corpora_strict
# !mkdir -p /content/gdrive/'My Drive'/saved_nnets/silver_standard

Создаем callback для сохранения сети

In [0]:
# checkpoint = ModelCheckpoint('/content/gdrive/My Drive/saved_nnets/char_rnn/demo-lstm-{epoch:02d}.hdf5')
# checkpoint = ModelCheckpoint('/content/gdrive/My Drive/saved_nnets/war_and_peace_h/war-and-peace-h-{epoch:02d}.hdf5')
# checkpoint = ModelCheckpoint('/content/gdrive/My Drive/saved_nnets/ru_corpora/ru-corpora-{epoch:02d}.hdf5')
checkpoint = ModelCheckpoint('/content/gdrive/My Drive/saved_nnets/ru_corpora_strict/ru-corpora-strict-2-{epoch:02d}.hdf5')
# checkpoint = ModelCheckpoint('/content/gdrive/My Drive/saved_nnets/silver_standard/silver-standard-{epoch:02d}.hdf5')

Создаём callback для своевременной приостановки обучения

In [0]:
early_stop = EarlyStopping(monitor='val_loss', verbose=1, mode='min')

## Запускаем обучение нейронной сети

In [0]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

In [0]:
model.load_weights('/content/gdrive/My Drive/saved_nnets/ru_corpora_strict/ru-corpora-strict-2-15.hdf5')

In [0]:
history = model.fit(x, y,
          batch_size=128,
          epochs=30,
          validation_split=0.2,
          initial_epoch=15,
          callbacks=[checkpoint,early_stop], verbose=1)

Train on 1378776 samples, validate on 344695 samples
Epoch 16/30
1378776/1378776 [==============================] - 842s 610us/sample - loss: 1.4265 - val_loss: 1.5077
Epoch 17/30
1378776/1378776 [==============================] - 848s 615us/sample - loss: 1.4217 - val_loss: 1.5015
Epoch 18/30
1378776/1378776 [==============================] - 851s 617us/sample - loss: 1.4176 - val_loss: 1.4998
Epoch 19/30
1378776/1378776 [==============================] - 846s 613us/sample - loss: 1.4133 - val_loss: 1.4991
Epoch 20/30
1378776/1378776 [==============================] - 838s 608us/sample - loss: 1.4098 - val_loss: 1.4955
Epoch 21/30
  27008/1378776 [..............................] - ETA: 13:08 - loss: 1.3756

## Загружаем обученную сеть

In [0]:
# model = create_model(context_length, len(chars))

In [0]:
# model.load_weights('/content/gdrive/My Drive/saved_nnets/char_rnn/demo-lstm-60.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/char_rnn/nietzsche-60.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/char_rnn/wap-40.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/war_and_peace/war-and-peace-2-23.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/ru_corpora/ru-corpora-008.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/ru_corpora_strict/ru-corpora-strict-18.hdf5')
# model.load_weights('/content/gdrive/My Drive/saved_nnets/silver_standard/silver-standard-10.hdf5')



## Применяем сеть для подстановки окончаний слов

Задаём условия, в которых будет производиться предсказание

In [0]:
sentence_length = 8  # Минимальная длина предложенй, учавствующих в проверке
initial_words = 4    # Количество опорных слов в начале предложения
ending_length = 2    # Длина окончания

Загружаем тестовую выборку

In [0]:
path = "/content/gdrive/My Drive/saved_nnets/test_sample.txt"
# !find . -type f -name "test_sample.strict.txt" -delete
# !ls
# files.upload()

In [0]:
with io.open(path, encoding='utf-8') as f:
    test_text = f.read().lower()
    
test_text[:50]

In [0]:
def predict_ending(base, word):
    for i in range(ending_length):
        x_pred = np.zeros((1, context_length, len(chars)))
        for t, char in enumerate(base):
            x_pred[0, t, char_indices[char]] = 1.

        prediction = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(prediction)
        next_char = indices_char[next_index]

        word += next_char
        base = base[1:] + next_char
        
    return word

In [0]:
# test_sample = []
# for sentence in test_text.split("\n"):
#     words = sentence[:-1].split(" ")
#     if (len(words) < sentence_length):
#         continue;
#     clipped = words[initial_words:]
    
#     for i, word in enumerate(clipped):
#         if (len(word) > ending_length):
#             clipped[i] = word[:-ending_length]

#     final_sentence = words[:initial_words] + clipped
#     test_sample.append(" ".join(final_sentence))

In [0]:
# l = len(test_sample)
# test_sample[0]

Предсказываем

In [0]:
# predicted_sample = []
# for sentence in test_sample:
#     words = sentence.split(" ")
#     base = " ".join(words[:initial_words])
#     clipped = words[initial_words:]
#     final_sentence = base
    
#     for word in clipped:
#         base = base + " " + word
#         base = base[-context_length:]
#         word = predict_ending(base, word)
        
#         final_sentence = final_sentence + " " + word
    
#     predicted_sample.append(final_sentence)
    

Второй вариант: предсказываем сразу в процессе считывания тестовой выборки. При таком подходе сеть не будет пытаться подобрать окончания к предлогам, союзам и другим словам, которые оказались короче заданной длинны.

In [0]:
predicted_sample = []
with io.open(path, encoding='utf-8') as f:
    for sentence in f.readlines():
        words = sentence[:-1].lower().split(" ")  #[:-2] - отрезаем ".\n" на конце
        if (len(words) < sentence_length):
          continue;
        clipped = words[initial_words:]
    
        for i, word in enumerate(clipped):
            if (len(word) > ending_length):
                word = word[:-ending_length]
                base = words[:initial_words] + clipped[:i]
                base = " ".join(base) + " " + word
                base = base[-context_length:]
                clipped[i] = predict_ending(base, word)
                
        final_sentence = words[:initial_words] + clipped
        predicted_sample.append(" ".join(final_sentence))

## Строим график функции ошибки

In [0]:
test_loss = history.history['val_loss']
for i in history.epoch:
  if test_loss[i] < test_loss[i+1]:
    print(i)
    break

In [0]:
# Get training and test loss histories
training_loss = history.history['loss']
test_loss = history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

Предложения с добавленными нейросетью окончаниями

In [0]:
for predicted in predicted_sample:
  print(predicted)

In [0]:
# На этом месте можно вручную прервать исполнение
files.upload()

## Применяем сеть для предсказания следующего символа

Начальная строка

In [0]:
# text_seed = "supposing that truth is a woman--what th"
text_seed = "он говорил на том изысканном французском"


Преобразуем строку в One Hot Encoding

In [0]:
x_pred = np.zeros((1, context_length, len(chars)))
for t, char in enumerate(text_seed):
    x_pred[0, t, char_indices[char]] = 1

In [0]:
x_pred.shape

In [0]:
x_pred[0,0]

Запускаем предсказание

In [0]:
prediction = model.predict(x_pred, verbose=0)

In [0]:
prediction

In [0]:
pred_index = np.argmax(prediction)

In [0]:
pred_index

In [0]:
pred_char = indices_char[pred_index]

In [0]:
pred_char

## Генерируем текст

In [0]:
sentence = text_seed
generated = ''

В цикле генерируем N симвлов

In [0]:
n_symbols = 100
for i in range(n_symbols):
    x_pred = np.zeros((1, context_length, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    prediction = model.predict(x_pred, verbose=0)[0]
    next_index = np.argmax(prediction)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

In [0]:
print(text_seed + generated)

## Генерируем текст с сэмплированием на основе "температуры"

Функция для изменения распределения вероятности символов в зависимости от "температуры"

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
sentence = text_seed
generated = ''

In [0]:
temperature = 0.4

In [0]:
for i in range(100):
    x_pred = np.zeros((1, context_length, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.

    prediction = model.predict(x_pred, verbose=0)[0]
    next_index = sample(prediction, temperature)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

In [0]:
print(text_seed + generated)

## Домашнее задание

В этом домашнем задании вам нужно сгенерировать интересный текст с помощью LSTM.

1. Выберите интересный источник для обучения (https://www.gutenberg.org/, http://lib.ru/ ).
2. Подберите архитектуру LSTM и количество эпох обучения.
3. Попробуйте использовать разную "температуру" при генерации текстов.
4. Лучший текст, который вам удастся получить, пришлите кураторам со ссылкой на ноутбук с кодом решения генерации этого текста.
